## Program Representation


Every Julia program starts life as a string. For example 

```julia
p = "print(`hello word`)"
```

is a program.

Strings are parsed into objects called expressions. An expression has the type **`Expr`**.

In [391]:
p = "print(`hello word`)"

"print(`hello word`)"

In [392]:
ex_p = parse(p)

:(print(@cmd("hello word")))

In [397]:
typeof(ex_p)

Expr

Expressions have 2 parts:

- A `Symbol` identifier that tell us the kind of expression it is.
- An array contaning the expression arguments which can be symbols, other expressions or literal values.

In [399]:
ex_p.head

:call

In [402]:
ex_p.args

2-element Array{Any,1}:
 :print              
 :(@cmd "hello word")

Expressions can also be defined as follows

In [408]:
ex2 = Expr(:call, :+, 1, 1)

:(1 + 1)

In [409]:
ex3 = :(1 + 1)

:(1 + 1)

# Symbolic expression manipulation


The most simple symbolic unit in Julia is a `Symbol`. We can define symbols using the `:` followed by a name.
For example `:house` is a `Symbol`.

In [23]:
a = :house

:house

In [68]:
typeof(a)

Expr

We can define `Symbol` objects using `Symbol()`.

In [69]:
?Symbol()

```
Symbol(x...) -> Symbol
```

Create a `Symbol` by concatenating the string representations of the arguments together.


In [70]:
:abc == Symbol("abc")

true

Symbols can be evaluated using the `eval` method.

In [71]:
x = 23
eval(:x)

23

In the context of an expression, symbols are used to indicate access to variables; when an expression is evaluated, a symbol is replaced with the value bound to that symbol in the appropriate scope.



## Expressions

Expressions are combinations of symbols, for example `:(x+y)` is a `Expr`.

Just like with symbols, we can evaluate an expresion using the `eval` function. Notice that, all symbols in the expression need to have a value. Otherwise we will not be able to evaluate the expression.

In [102]:
workspace()

In [130]:
a = :(x+y)

:(x + y)

In [104]:
typeof(a)

Expr

In [105]:
# This will fail since x and y have no values
eval(a)

LoadError: [91mUndefVarError: x not defined[39m

In [109]:
x = 2; y = 10

10

In [153]:
# Now it will not fail because the expression a = :(x+y) depends on symbols whose values
# are defined
eval(a)

true

### Substitutions inside an expression

 
We can substitute a variable by it's value inside an expression. 

##### Example

Let us conside the expression `:(aux = some_variable + 2)`.
This expression can allways be creater, independently of whether variable `some_variable` exist in the namespace or not. Nevertheless `:(aux = $some_variable + 2)` can only be created if `some_variable` exists.


```julia
workspace()
:(aux = some_variable + 2)
```
```
:(aux = some_variable + 2)
```

In [201]:
workspace()
:(aux = some_variable + 2)

:(aux = some_variable + 2)

In [205]:
# This will not work
workspace()
:(aux = $some_variable + 2)

LoadError: [91mUndefVarError: some_variable not defined[39m

In [207]:
workspace()
some_variable = 3
:(aux = $some_variable + 2)

:(aux = 3 + 2)

Another example

In [222]:
workspace()
height = 4
a1 = :(x < $height < y)

:(x < 4 < y)

In [225]:
a2 = :(x < height < y)

:(x < height < y)

Let us evaluate the expressions 

In [227]:
x = 1 ; y = 10

10

In [228]:
eval(a1), eval(a2)

(true, true)

In [229]:
## Notice that if now we change the value in `height` then the second expression will be evaluated to false.
## The first expression stays true because it was created with a 4 inside.
height = 60
eval(a1), eval(a2)

(true, false)

### Fields of an expression

All expressions contain 3 fields:

 - **`:head`**: Type `Symbol`. Gives some high level information about the "type" of expression
 - **`:args`**: Type `Array`. Contains all `symbols` used in the expression.
 - **`:typ` **: Type `Any`. 
 

In [251]:
height = 4
a = :(x < $height < y)

:(x < 4 < y)

In [252]:
eval(a)

true

In [253]:
fieldnames(a)

3-element Array{Symbol,1}:
 :head
 :args
 :typ 

In [254]:
a.head

:comparison

In [276]:
a.args

5-element Array{Any,1}:
  :x
  :<
 4  
  :<
  :y

In [277]:
a.typ

Any

## More complex expressions between `quote` and `end`.

You can build more complex expressions (actually any arbitrary piece of julia code) between `quote` and `end`.


In [278]:
ex = quote
   a = x + 1
   a = 2 * a
end

quote  # In[278], line 2:
    a = x + 1 # In[278], line 3:
    a = 2a
end

In [279]:
typeof(ex)

Expr

Expressions are represented in a tree form. 

In [283]:
ex.head

:block

In [284]:
ex.args

4-element Array{Any,1}:
 :( # In[278], line 2:)
 :(a = x + 1)          
 :( # In[278], line 3:)
 :(a = 2a)             

In [285]:
a.typ

Any

In [286]:
ex.args[1]

:( # In[278], line 2:)

In [287]:
ex.args[2]

:(a = x + 1)

In [288]:
ex.args[3]

:( # In[278], line 3:)

In [289]:
ex.args[4]

:(a = 2a)

### Expressions from strings

We can convert an string containing julia code to an expression using **`parse`**.



In [293]:
ex3 = parse("(4 + 4) / 2")

:((4 + 4) / 2)

In [294]:
ex3

:((4 + 4) / 2)

# Getting inside expressions: Expression manipulation

We can go inside an expression and change it

In [323]:
a = :(x+y)

:(x + y)

In [346]:
a.args

3-element Array{Any,1}:
 :+   
 :(2x)
 :y   

In [326]:
a.args[2] = :(2*x)

:(2x)

In [327]:
a

:(2x + y)

#### Modify functions


The following example creates an expression for a function `f` and returns and modifies it.

In [351]:
ex_f = :(f(x) = x^2)

:(f(x) = begin  # In[351], line 1:
            x ^ 2
        end)

In [352]:
eval(ex_f)(2)

4

In [353]:
length(ex_f.args)

2

In [354]:
ex_f.args[1]

:(f(x))

In [355]:
ex_f.args[2]

quote  # In[351], line 1:
    x ^ 2
end

In [360]:
typeof(ex_f.args[2])

Expr

We have seen that the expression `ex_f` has another expression inside.

In [364]:
sub_ex = ex_f.args[2]

quote  # In[351], line 1:
    x ^ 2
end

In [365]:
sub_ex.args

2-element Array{Any,1}:
 :( # In[351], line 1:)
 :(x ^ 2)              

In [366]:
sub_ex.args[2]

:(x ^ 2)

In [369]:
sub_ex.args[2].args

3-element Array{Any,1}:
  :^
  :x
 2  

In [370]:
sub_ex.args[2].args[3] = 3

3

In [371]:
ex_f

:(f(x) = begin  # In[351], line 1:
            x ^ 3
        end)

In [372]:
eval(ex_f)(3)

9

In [377]:
?dump

search: dump randjump reducedim mapreducedim module_name



```
dump(x)
```

Show every part of the representation of a value.


In [376]:
dump(ex_f)

Expr
  head: Symbol =
  args: Array{Any}((2,))
    1: Expr
      head: Symbol call
      args: Array{Any}((2,))
        1: Symbol f
        2: Symbol x
      typ: Any
    2: Expr
      head: Symbol block
      args: Array{Any}((2,))
        1: Expr
          head: Symbol line
          args: Array{Any}((2,))
            1: Int64 1
            2: Symbol In[351]
          typ: Any
        2: Expr
          head: Symbol call
          args: Array{Any}((3,))
            1: Symbol ^
            2: Symbol x
            3: Int64 3
          typ: Any
      typ: Any
  typ: Any


### Simplification of expressions

In [ ]:
ex = :(X = W*x + b)

In [ ]:
typeof(ex.args[1])

In [ ]:
ex.args[2]

### Functions on Expressions

A useful feature of Julia is the capability to generate and manipulate Julia code within Julia itself.

We have already seen one example of a function returning `Expr` objects: the `parse()` function, which takes a string of Julia code and returns the corresponding Expr. A function can also take one or more `Expr` objects as arguments, and return another `Expr`. Here is a simple, motivating example:


In [410]:
function math_expr(op, op1, op2)
    expr = Expr(:call, op, op1, op2)
    return expr
end

math_expr (generic function with 1 method)

In [411]:
ex1 = math_expr(:+, 1, Expr(:call, :*, 4, 5))

:(1 + 4 * 5)

In [416]:
eval(ex1)

21

In [413]:
ex2 = math_expr(:+, 1, Expr(:call, :-, 4, 1))

:(1 + (4 - 1))

In [422]:
eval(ex2)

4

As another example, here is a function that doubles any numeric argument, but leaves expressions alone:



In [418]:
function make_expr2(op, opr1, opr2)
           opr1f, opr2f = map(x -> isa(x, Number) ? 2*x : x, (opr1, opr2))
           retexpr = Expr(:call, op, opr1f, opr2f)
           return retexpr
       end

make_expr2 (generic function with 1 method)

In [419]:
make_expr2(:+, 1, 2)

:(2 + 4)

In [420]:
ex = make_expr2(:+, 1, Expr(:call, :*, 5, 8))

:(2 + 5 * 8)

In [423]:
eval(ex)

42

In [425]:
ex = make_expr2(:+, Expr(:call, :*, 5, 8), 30)

:(5 * 8 + 60)

In [426]:
eval(ex)

100